In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

import spacy
nlp = spacy.load('en')

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', batch_first = True)
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:09<00:00, 8.86MB/s]


In [3]:
TEXT.build_vocab(train_data, max_size = 10000,vectors = "glove.6B.300d", unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:31, 2.20MB/s]                           
100%|█████████▉| 399356/400000 [00:38<00:00, 10749.79it/s]

In [0]:
BATCH_SIZE = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = embedding_dim, out_channels = 150, kernel_size = fs)for fs in [4,5]])
        self.conv2 = nn.Conv1d(in_channels = 1, out_channels = 100, kernel_size = 3)              
        self.fc1 = nn.Linear(200, 100) 
        self.fc2=nn.Linear(100,1)
        
    def forward(self, text):

        embedded = self.embedding(text)        
        embedded = embedded.permute(0, 2, 1)

        sent_len=embedded.size(2)
        padding=3000-sent_len
        batch_size=embedded.size(0)
        torch_padding=torch.zeros(batch_size,300,padding,dtype = embedded.dtype,device = embedded.device)
        lz=[embedded,torch_padding]
        zcat = torch.cat(lz, dim = 2)

        conved = [F.relu(conv(zcat)) for conv in self.convs]

        pooled=[]
        for c in conved:
          pooled.append(F.max_pool1d(c,c.shape[2]))

        pooled = [f.permute(0,2,1) for f in pooled]

        pooled2 = [F.max_pool1d(p, 2) for p in pooled]

        pooled3 = [F.relu(self.conv2(p1)) for p1 in pooled2]

        pooled4=[]
        for c in pooled3:
            pooled4.append(F.max_pool1d(c,c.shape[2]))

        final = torch.cat(pooled4,dim = 1)
        final = final.reshape(batch_size,200)
        full1 = self.fc1(final)
        full2= self.fc2(full1)
        return full2


In [8]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
model = CNN1d(INPUT_DIM, EMBEDDING_DIM, DROPOUT, PAD_IDX)

10002 1


In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,426,501 trainable parameters


In [10]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data.shape)

embedding.weight torch.Size([10002, 300])
convs.0.weight torch.Size([150, 300, 4])
convs.0.bias torch.Size([150])
convs.1.weight torch.Size([150, 300, 5])
convs.1.bias torch.Size([150])
conv2.weight torch.Size([100, 1, 3])
conv2.bias torch.Size([100])
fc1.weight torch.Size([100, 200])
fc1.bias torch.Size([100])
fc2.weight torch.Size([1, 100])
fc2.bias torch.Size([1])


In [11]:
print(model)
print(dir(model))

CNN1d(
  (embedding): Embedding(10002, 300, padding_idx=1)
  (convs): ModuleList(
    (0): Conv1d(300, 150, kernel_size=(4,), stride=(1,))
    (1): Conv1d(300, 150, kernel_size=(5,), stride=(1,))
  )
  (conv2): Conv1d(1, 100, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=200, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)
['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_forward_hooks', '_forward_pre_hooks', '_get_name', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_modules', '_named_members', '_parameters', '_register_load_state_dict_pre_hook',

In [0]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [0]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    #Returns accuracy per batch, i.e. for 8/10 right, this returns 0.8 #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train() #to ensure the dropout is "turned on" while training.
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval() #to ensure the dropout is "turned off" while evaluating.
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [20]:
N_EPOCHS = 10
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './sentimentModel.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 19s
	Train Loss: 0.589 | Train Acc: 65.21%
	 Val. Loss: 0.409 |  Val. Acc: 81.55%
Epoch: 02 | Epoch Time: 1m 18s
	Train Loss: 0.345 | Train Acc: 85.33%
	 Val. Loss: 0.357 |  Val. Acc: 84.04%
Epoch: 03 | Epoch Time: 1m 18s
	Train Loss: 0.219 | Train Acc: 91.59%
	 Val. Loss: 0.352 |  Val. Acc: 85.33%
Epoch: 04 | Epoch Time: 1m 18s
	Train Loss: 0.116 | Train Acc: 95.87%
	 Val. Loss: 0.458 |  Val. Acc: 84.39%
Epoch: 05 | Epoch Time: 1m 18s
	Train Loss: 0.049 | Train Acc: 98.26%
	 Val. Loss: 0.613 |  Val. Acc: 84.64%
Epoch: 06 | Epoch Time: 1m 19s
	Train Loss: 0.029 | Train Acc: 98.95%
	 Val. Loss: 0.804 |  Val. Acc: 84.21%
Epoch: 07 | Epoch Time: 1m 19s
	Train Loss: 0.021 | Train Acc: 99.23%
	 Val. Loss: 0.787 |  Val. Acc: 84.87%
Epoch: 08 | Epoch Time: 1m 19s
	Train Loss: 0.026 | Train Acc: 99.12%
	 Val. Loss: 0.879 |  Val. Acc: 84.32%
Epoch: 09 | Epoch Time: 1m 18s
	Train Loss: 0.014 | Train Acc: 99.50%
	 Val. Loss: 0.922 |  Val. Acc: 84.47%
Epoch: 10 | Epoch T

In [21]:
model.load_state_dict(torch.load('./sentimentModel.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.351 | Test Acc: 85.05%


In [0]:
def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [28]:
predict_sentiment(model, "This is undoubtedly the best movie ever!!!")

0.9935504198074341

In [26]:
predict_sentiment(model, "I have to give this zero stars")

0.01589825190603733

In [27]:
predict_sentiment(model, "Good at being terrible ")

0.005191510077565908